LBM 2D implementation.


In [52]:
import numpy as np
import matplotlib.pyplot as plt; 
import os

# Discretization

First thing we need to do for blood flow simulaion in frame of **LBM** - 
perform a proper discretization of physizcal system. 
In this case I used information from 
[first link](http://wiki.palabos.org/_media/howtos:lbunits.pdf) 
and section 5 from 
[second link](http://mafija.fmf.uni-lj.si/seminar/files/2012_2013/Lattice_Boltzmann_method.pdf).

Main idea is that we need to choose a physical value, which characterize the modeling system and this value must stay the same during the discretization procedure.

It is obvious that the simplest approach is to use a dimensionless value - Reynolds number.
So in each system during the discretization **Reynolds number must be the same**.

### 1.Physical system.

Modeling data are based on the folowing [table](http://www.coheadquarters.com/PennLibr/MyPhysiology/lect5/table5.01.htm):
1. Vein diameter: $l_{phys}$ = 0.5 [cm.]
2. Blood flow velocity in vein: $v_{phys}$= 10 [cm./sec.]
3. Blood viscosity: $\nu = 2.5^{-2}$ [$cm^{2}$/sec.]

Reynolds number in **Physical system**:

\begin{equation}
    Re_{phys} = \frac{l_{phys}v_{phys}}{\nu_{phys}}
\end{equation}

In [53]:
# Physical values
l_phys = 0.5
v_phys = 10.0
nu_phys = 2.5e-2

print('Characteristic len: {0}'.format(l_phys))
print('Characteristic vel: {0}'.format(v_phys))
print('Characteristic vis: {0}'.format(nu_phys))
print('Physical Re = ' + str(l_phys * v_phys / nu_phys))
print('---' * 10)

Characteristic len: 0.5
Characteristic vel: 10.0
Characteristic vis: 0.025
Physical Re = 200.0
------------------------------


### 2. Dimensionless system.

The main idea of this intermidiate discretization system is that ** $l_{dim}$ and $v_{dim}$ are equal to 1** . 

So we need to calulate scaling factors for each of them: 
1. Dimensionless length: $l_{dim} = \frac{l_{phys}}{L_{0}} = 1$ which leads to $L_{0} = l_{phys}$.
2. Dimensionless velocity: $v_{dim} = \frac{v_{phys}}{V_{0}} = 1$ which leads to $V_{0} = v_{phys}$.
3. Dimensionless viscosity: $\nu_{dim} = \frac{\nu_{phys}}{L_{0}V_{0}}$.

Reynolds number in **Dimensionless system**:

\begin{equation}
    Re_{dim} = \frac{l_{dim}v_{dim}}{\nu_{dim}} = [l_{dim} = v_{dim} = 1] = \frac{1}{\nu_{lbm}}
\end{equation}

In [54]:
# Dimensionless values
L0 = l_phys / 1.
V0 = v_phys / 1.
nu_dim = nu_phys / (L0 * V0)

print('Dimensionless Re = ' + str(1. * 1. / nu_dim))
print('---' * 10)

Dimensionless Re = 200.0
------------------------------


### 3. LBM system.

In this system we need to define:
- Space step $dx$ could be obtained as value inversery propotional to number of nodes on 1 unit length of modeling area.
- Time step $dt = dx^{2}$, for mode information you can see this [link](http://wiki.palabos.org/_media/howtos:lbunits.pdf)

After choosing values listed above in appropriate way we need to define velocity and check Reynolds number:
1. LBM velocity $v_{lbm} = \frac{dt}{dx} \dot v_{dim}$
2. LBM viscosity $\nu_{lbm} = \frac{dt}{dx^{2}} \dot \nu_{dim}$

Reynolds number in **Dimensionless system**:

\begin{equation}
    Re_{lbm} = \frac{N v_{lbm}}{\nu_{lbm}}
\end{equation}

In [55]:
# LBM values
N = 100 # Number of cells on physical length l_phys
dx = 1./N
dt = dx**2
u_lbm = dt/dx * 1.
nu_lbm = dt/dx**2 * nu_dim

print('Time step dt = {0}'.format(dt))
print('Space step dx = {0}'.format(dx))
print('u_lbm = {0}'.format(u_lbm))
print('nu_lbm = {0}'.format(nu_lbm))
print('LBM Re = {0}'.format(N*u_lbm/ nu_lbm))
print('---' * 10)

Time step dt = 0.0001
Space step dx = 0.01
u_lbm = 0.01
nu_lbm = 0.005
LBM Re = 200.0
------------------------------


At this stage we obtain all necessary values for further **LBM** calculations.
In this example we use **D2Q9** velocity template, which means that we perform calculations in two dimensional space with nine possible directions of pseudo-particles movement 
(for more information you could watch [article](http://www.mathnet.ru/links/5df81992c52b0a5b1a53b524ba3028dc/vmp280.pdf)).

Below we setup modeling area geometry, physics parameters and constans (like weights for equilibrium distribution function calculation, e.t.c.) which are necessary for **LBM** calculations:
1. Size of modeling domain 
2. Thrombus center position and possible radiuses (here we assume that thrombus is a half of sphere).
3. Initial blood parameters, like maximum velocity, viscosity, Reynolds number and relaxation parameter. This values were calulcated previously during discretization.

In [56]:
# Modeling domain parameters
nx = 1000; ny = 50;
# Obstacle (thrombus) position and radius
cx = round(nx // 4); cy = 0; r = 33;

# Physical parameters of blood flow
uLB = u_lbm
nulb = nu_lbm; 
Re = N * uLB / nulb

# Relaxation parameter
omega = 1.0 / (3.* nulb + 0.5);

# Number of discrete velocities in model (D2Q9)
q = 9 
# Normal directions of velocities for D2Q9 velocity template
c = np.array([(x,y) for x in [0, -1 ,1] for y in [0, -1, 1]]) 

# Weights for further f equilibrium calculations
t = 1./36. * np.ones(q)
t[np.asarray([np.linalg.norm(ci) < 1.1 for ci in c])] = 1./9.; t[0] = 4./9.

# Opposite directions of velocities for D2Q9 velocity template
# Necessary for No-Slip boundary condidtions implementation (bounce-back)
noslip = [c.tolist().index((-c[i]).tolist()) for i in range(q)] 

i1 = np.arange(q)[np.asarray([ci[0] <  0  for ci in c])] # Unknown on right wall.
i2 = np.arange(q)[np.asarray([ci[0] == 0  for ci in c])] # Vertical middle.
i3 = np.arange(q)[np.asarray([ci[0] >  0  for ci in c])] # Unknown on left wall.

print('Reynolds number = {0}'.format(Re))
print('Domain size: [{0}x{1}]'.format(nx*dx,ny*dx))
print('---' * 10)

Reynolds number = 200.0
Domain size: [10.0x0.5]
------------------------------


When all necessary parameters are set up, we need to define some functions for further calculations and obstacle (thrombus) further setup.

In case of escaping of misunderstandings I will tell a few words about obstacle implementation.
We set up two dimensional boolean array in which:
- Node value is true if this node represents obstacle
- Otherwise node value represents fluid (value is equal to false)

And after **streaming** and **collision** execution we performs No-Slip (bounce-back) boundary conditions for all nodes with true value. 
This helps us achieve obstacle effects.

In [57]:
# Helper function for density computation
sumpop = lambda fin: np.sum(fin,axis=0) 

# Equilibrium distribution function
def equilibrium(rho, u):              
    cu   = 3.0 * np.dot(c, u.transpose(1, 0, 2))
    usqr = 3./ 2. * (u[0]**2 + u[1]**2)
    feq = np.zeros((q,nx,ny))
    for i in range(q): 
        feq[i,:,:] = rho*t[i]*(1.+cu[i]+0.5*cu[i]**2-usqr)
    return feq

# Set Up thrombus    
def form_tromb(obst, thr_radius):
    for x in range(cx-r-1, cx+r+1):
        for y in range(0,ny):
            if((x-cx)**2 + (y-cy)**2 < thr_radius**2):
                obst[x,y] = True

In [58]:
# For colorbar configurations
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Visualisation of modelig area
def contour_plot(time, val, points, val_name, x_min, x_max, y_min, y_max, thr_radius):
    fig = plt.figure(figsize = (12, 3))
    ax = plt.subplot(111)
    
    ax.set_xlabel(u'X-координата.', fontsize = 15)
    ax.set_ylabel(u'Y-координата.', fontsize = 15)
    
    x = np.arange(x_min, x_max)
    y = np.arange(y_min, y_max)
    
    val_tr = val.T[y_min:y_max, x_min:x_max]
    
    cont = plt.contourf(x, y, val_tr, cmap=plt.cm.jet) # np.arange(0.0, 0.08, 0.005)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    b = plt.colorbar(cont, orientation='vertical',cax=cax)
    
    obstacle = plt.Circle((cx, cy), thr_radius, color='k')
    ax.add_artist(obstacle)
    
    # print measurement points
    for pair in points:
        ax.plot(pair[0], pair[1], 'kx', mew=2, ms=5)
    plt.show()    

In [59]:
def streamline_plot(time, v, points, x_min, x_max, y_min, y_max, thr_radius):
    fig = plt.figure(figsize = (12, 3))
    ax = plt.subplot(111)
    plt.axis([x_min,x_max,y_min,y_max])
    
    ax.set_xlabel(u'X-координата.', fontsize = 15)
    ax.set_ylabel(u'Y-координата.', fontsize = 15)
    
    x = np.arange(x_min, x_max)
    y = np.arange(y_min, y_max)
    
    vx = v[0].T[y_min:y_max,x_min:x_max]
    vy = v[1].T[y_min:y_max,x_min:x_max]
    
    strm = plt.streamplot(x, y, vx, vy, density = [0.6, 1], linewidth=2, color='#1f77b4')  
    
    obstacle = plt.Circle((cx, cy), thr_radius, color='k')
    ax.add_artist(obstacle)
    # print measurement points
    for pair in points:
        ax.plot(pair[0], pair[1], 'kx', mew=2, ms=5)
    #plt.show()
    #plt.savefig('lol.png')

Below we setup the points position in which we will obtain density fluctuations. 

The area in which poits located could be setup like this:
- Area is a rectungle and all points for measurements are inside this rectangle
- Setup TOP LEFT corner of rectungle
- Setup width and lenght of rectungle
- Choose number of nodes along *x*(*y*)-directions

In [60]:
top_x = cx + r   # Right next to thrombus
top_y = 45
len_x = 4 * r
len_y = top_y
num_p_x = 5
num_p_y = 5

def set_points_pos(top_x, top_y, len_x, len_y, num_p_x, num_p_y):
    if top_x + len_x > nx:
        print('Error top_x + len_x > nx')
        return
    
    if top_y - len_y < 0:
        print('Error top_y - len_y < 0')
        return
    
    x_step = len_x / num_p_x
    X = np.array([int(top_x + i * x_step)  for i in range(num_p_x)])
    
    y_step = len_y / num_p_y
    Y = np.array([int(top_y - i * y_step)  for i in range(num_p_y)])
    
    return np.array([[x,y] for x in X for y in Y])

points = set_points_pos(top_x, top_y, len_x, len_y, num_p_x, num_p_y)

print('Mesurement points\n')
for i in range(len(points)):
    print('{0} : {1}'.format(i, points[i]))
print('---' * 10)
print(type(top_x), type(cx))

Mesurement points

0 : [283  45]
1 : [283  36]
2 : [283  27]
3 : [283  18]
4 : [283   9]
5 : [309  45]
6 : [309  36]
7 : [309  27]
8 : [309  18]
9 : [309   9]
10 : [335  45]
11 : [335  36]
12 : [335  27]
13 : [335  18]
14 : [335   9]
15 : [362  45]
16 : [362  36]
17 : [362  27]
18 : [362  18]
19 : [362   9]
20 : [388  45]
21 : [388  36]
22 : [388  27]
23 : [388  18]
24 : [388   9]
------------------------------
<class 'int'> <class 'int'>


Thrombus status is value which will be used further as target value of NW prediction.

Thrombus status could take three different values, depending on thrombus size:
1. value 0 corresponds to **little thrombus** ($ r_{thr} < 0.3D_{vein} $)
2. value 1 corresponds to **medium thrombus** ($ 0.3D_{vein} < r_{thr} < 0.6D_{vein}$)
3. value 2 corresponds to **big thrombus** ($r_{thr} > 0.6D_{vein}$)

In [61]:
import csv
import pandas as pd

# Create folder for output dataset storage
out_folder = 'lbm_out_data'
out_file_train = os.path.join(out_folder, 'lbm_train_data1.txt' )
out_file_test = os.path.join(out_folder, 'lbm_test_data1.txt' )
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

# Obtain thrombus status
thrombus_staus = 0 # in case r <= 0.3*ny
if r > 0.3*ny and r <= 0.6*ny:
    thrombus_staus = 1
elif r > 0.6*ny:
    thrombus_staus = 2

out_data = []

print('Thrombus status: {0}'.format(thrombus_staus))
print('---' * 10)

Thrombus status: 2
------------------------------


Implementation of two dimensional (**D2Q9**) **LBM** solver in accordance with 
[article](http://www.mathnet.ru/links/15cb7cadb3fc62a1fbe75e4b6f67539e/vmp280.pdf).

In [64]:
import time
%matplotlib inline
# Number of iterations
maxIter = 220000
# Display launch parameters
print('Launch parameters:')
print('\tDomain size: [{0}x{1}]'.format(nx, ny))
print('\tSpace step dx = {0}'.format(dx))
print('\tTotal time: {0}'.format(maxIter * dt) + ' sec.')
print('\tTime step dt = {0}'.format(dt))

print('\tMaximum velocity = {0}'.format(u_lbm))
print('\tBlood viscosity = {0}'.format(nu_lbm))
print('\tReynolds number = {0}'.format(N*u_lbm/ nu_lbm))
print('----'*10)
#print(type(r), type(obstacle))


# Main loop starts

# Create obstacle
obstacle = np.fromfunction(lambda x,y: x == -1, (nx,ny))
# Obstacle on top and bottom boundary of modeling area
obstacle[0:nx,0].fill(True)
obstacle[0:nx,ny-1].fill(True)
# Thrombus obstacle
form_tromb(obstacle, r)

# Input on left boundary - velocity
vin = np.zeros((2,nx,ny))
v_parabola_profile = np.array([uLB * np.sin(np.pi * y / (ny - 1) ) for y in range(0, ny)])
vin[0,:,:] = v_parabola_profile

feq = equilibrium(1.0, vin); 
fin = feq.copy()

# Clear previous data and send trombus status
out_data[:] = []
out_data.append([thrombus_staus for i in range(len(points))])

start_t = time.time()
for cur_time in range(maxIter):
    # Macroscopic parameters update
    rho = sumpop(fin)
    u = np.dot(c.transpose(), fin.transpose((1,0,2))) / rho

    # Left wall: Zou/He boundary condition.
    u[:,0,:] = vin[:,0,:]
    rho[0,:] = 1./(1.-u[0,0,:]) * (sumpop(fin[i2,0,:])+2.*sumpop(fin[i1,0,:]))
    feq = equilibrium(rho,u)
    fin[i3,0,:] = fin[i1,0,:] + feq[i3,0,:] - fin[i1,0,:]
    
    # Collision step.
    fout = fin - omega * (fin - feq)
    
    # No-Slip boundary conditions
    for i in range(q): 
        fout[i,obstacle] = fin[noslip[i],obstacle]
    
    # Streaming
    for i in range(q):
        fin[i,:,:] = np.roll(np.roll(fout[i,:,:],c[i,0],axis=0),c[i,1],axis=1)
    
    # Data collecting
    if cur_time > 80000 and cur_time % 10==0:
        print('Current time: {0}'.format(cur_time * dt))
        #v = np.sqrt(u[0]**2 + u[1]**2)
        
        #streamline_plot(time, u, points, cx-4*r, cx+7*r, 0, ny, r)
        #contour_plot(time, v, points, 'vel', cx-4*r, cx+7*r, 0, ny, r)
        out_data.append([rho[pair[0], pair[1]] for pair in points])
        
end_t = time.time()
print('Elapsed time T = {0}'.format(end_t - start_t))
        
# Write data for further NW calculations
with open(out_file_test, 'a') as out:
    writer = csv.writer(out, delimiter =",", lineterminator='\n')
    for line in map(list, zip(*out_data)):
        writer.writerows([line])
print('lol')

Launch parameters:
	Domain size: [1000x50]
	Space step dx = 0.01
	Total time: 22.0 sec.
	Time step dt = 0.0001
	Maximum velocity = 0.01
	Blood viscosity = 0.005
	Reynolds number = 200.0
----------------------------------------
Current time: 8.001000000000001
Current time: 8.002
Current time: 8.003
Current time: 8.004
Current time: 8.005
Current time: 8.006
Current time: 8.007
Current time: 8.008000000000001
Current time: 8.009
Current time: 8.01
Current time: 8.011000000000001
Current time: 8.012
Current time: 8.013
Current time: 8.014000000000001
Current time: 8.015
Current time: 8.016
Current time: 8.017000000000001
Current time: 8.018
Current time: 8.019
Current time: 8.02
Current time: 8.021
Current time: 8.022
Current time: 8.023
Current time: 8.024000000000001
Current time: 8.025
Current time: 8.026
Current time: 8.027000000000001
Current time: 8.028
Current time: 8.029
Current time: 8.030000000000001
Current time: 8.031
Current time: 8.032
Current time: 8.033000000000001
Current